In [7]:
# VARS
#-> Repository
input_folder = "../data_input/"
output_folder = "../data_output/"
param_folder = "../../_parameters/"
#-> MONGO
use_mongo=True
use_healthchecks=True
credentials = param_folder + 'mongodb.json'
db_src = 'covid-19-dev'
db_app = 'app-wsr-dev'
#-> Other
logo_cs = input_folder + "cashstory_all_white.png"
logo_cs = "https://bob.cashstory.com/assets/images/cashstory_all_white.png"

In [8]:
import bob
import json
from datetime import datetime, timedelta, date
import time
import pandas as pd
import openpyxl
import numpy as np
from pymongo import ASCENDING, DESCENDING
import os.path
from os import path
import plotly.express as px
import sys

#--- STEP0 --- : Init

#-> Date
now = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
today = date.today()
yesterday = (today - timedelta(days = 1))
b_yesterday = (today - timedelta(days = 2))

dbCon = None
MongoDB = None
if use_mongo:
    with open(credentials) as f:
        dbCon = json.load(f)
    mongo_host = dbCon["host"]
    mongo_port = dbCon["port"]
    mongo_username = dbCon["username"]
    mongo_password = dbCon["password"]
    #-> Bob connect
    MongoDB = bob.mongo.connect(mongo_host,mongo_port,mongo_username,mongo_password)

Successfully connected to MongoDB


In [ ]:
#--- Format ref France
ref_france = pd.read_excel(input_folder + 'REF_WSR.xlsx', sheet_name = 'REF_FRANCE')
ref_france['DEP_CODE'] = ref_france['DEP_CODE'].astype(str)
ref_france['DEP_NAME'] = ref_france['DEP_NAME'].astype(str)
ref_france['DEP_FULL'] = ref_france['DEP_CODE'] + ' - ' + ref_france['DEP_NAME']
# ref_france

In [9]:
#-- Map chart
#US - 500K
json_file = input_folder + 'usa_states.json'
with open(json_file) as f:
    states = json.load(f)
    
#US - 20m
json_file = input_folder + 'usa_states_20m.json'
with open(json_file) as f:
    states20m = json.load(f)
    
#US - 5m
json_file = input_folder + 'usa_states_5m.json'
with open(json_file) as f:
    states5m = json.load(f)
    
#France - Region
json_file = input_folder + 'france-region.json'
with open(json_file) as f:
    france_reg = json.load(f)
    
#France - Region
json_file = input_folder + 'france-departement.json'
with open(json_file) as f:
    france_dep = json.load(f)
    
#World map
json_file = input_folder + 'world-med.json'
with open(json_file) as f:
    worldmap = json.load(f)
    
def analysis(df,variable):
    Min = df[variable].min()
    Max = df[variable].max()
    Avr = round(df[variable].mean(),0)
    Med = round(np.median(df[variable]),0)
    AvrMax = round((Avr / Max),2)
    MedMax = round((Med / Max),2)
    to_print = f'{variable} : Min: {Min}, Max: {Max}, Average: {Avr}, AvrMax: {AvrMax}, Med: {Med}, MedMax: {MedMax}'
    return print(to_print)
    
#Fonction to add CSS
def updateChartCss(filename, css_filename):
    map = None
    css = None
    with open(filename) as f:
        map = f.read()

    with open(css_filename) as f:
        css = f.read()
    if (map.find('id="cs_css"') != -1):
        print("to do")
    else:
        result = map.replace("<body>",f'<body><style id="cs_css">{css}</style>')
        with open(filename, "w") as f:
            f.write(result)
            f.close()

In [10]:
#-- Datasource : Check if url exists and return df
# default seperator = ','
def check_url(url,separator=','):
    df = pd.DataFrame()
    try:
        df = pd.read_csv(url, sep=separator)
    except Exception as e:
        print(f'Error connecting to {url}.')
        print(e.__doc__)
        print(str(e)) 
    return df

In [11]:
#-- Datasource local : open csv in local if path exists or open DB MONGO
def get_datasource(src_name):
    src_path = output_folder + src_name + '.csv'
    if path.exists(src_path):
        df= pd.read_csv(src_path, sep=';',low_memory=False)
        if len(df) > 0 and 'Unnamed: 0' in df.columns:
            df = df.drop('Unnamed: 0',axis=1)
    else:
        df= bob.mongo.read_df(src_name,db_src)
        if len(df) > 0:
            df= df.drop('_id', axis=1)
        else:
            print(f'Warning ! Source {src_name} does not exist in csv or in DB')
    return df

In [12]:
#-- Trend : Create date scenario
#col scenario default : 'DATE_SCENARIO'
def get_lastdays(df,limit,title,col='DATE_SCENARIO'):
    df_b = pd.DataFrame()
    if 'DATE_ORDER' in df.columns:
        all_dates = df['DATE_ORDER'].sort_values(ascending=False).drop_duplicates().values.tolist()[:limit]
    for d in all_dates:
        tmp_df = df[df['DATE_ORDER'] == d]
        df_b = df_b.append(tmp_df)
        df_b[col] = title
    return df_b